In [ ]:
# Remove any generated glue code
!find . -mindepth 1 ! \( -name "dam_break.ipynb" -o -name "SWESolver.cpp" -o -name "exahype.log-filter" \) -exec rm -rf {} +

In [ ]:
import peano4, exahype2

In [ ]:
"""
Here, we define an ExaHyPE 2 solver, specifically an ADER-DG solver.
  It takes as arguments a name, a polynomial order,
  the unknowns, and the allowable cell sizes for the refinement.
"""
unknowns = {"h": 1, "hu": 2, "b": 1},
auxiliary_variables = {}

my_solver = exahype2.solvers.aderdg.rusanov.GlobalAdaptiveTimeStep(
    name                  = "SWESolver",
    order                 = 5,
    min_cell_h            = 0.008
    max_cell_h            = 0.1,
    time_step_relaxation  = 0.9,
    unknowns              = unknowns,
    auxiliary_variables   = auxiliary_variables
)

my_solver.plot_description = ", ".join(unknowns.keys())

In [ ]:
"""
With our solver defined, we can let it know what our equation will look like.
  "exahype2.solvers.PDETerms.User_Defined_Implementation" lets the project know
  that we will manually define the terms in an implementation file.
"""
my_solver.set_implementation(
    flux = exahype2.solvers.PDETerms.User_Defined_Implementation,
    ncp  = exahype2.solvers.PDETerms.User_Defined_Implementation,
    refinement_criterion = exahype2.solvers.PDETerms.User_Defined_Implementation
)

In [ ]:
"""
Here, we define an ExaHyPE 2 project to which our solver will be bound.
  This takes a namespace for the C++ code and a name for the executable.
  We then add our solver to this project and, just for cleanliness, set an output
  path for our results.
"""

exahype2_project = exahype2.Project(
    namespace     = ["exahype2", "training", "swe"],
    directory     = ".",
    project_name  = "DamBreak",
    executable    = "DamBreak"
)

exahype2_project.add_solver(my_solver)
exahype2_project.set_output_path("solutions")

In [ ]:
"""
Now that we have an ExaHyPE 2 project, let's populate it with some information:
  This is essentially the domain our solver will run in.
  The project needs to know the domain size, how long the simulation should run,
  how often it should plot the solution, and what type of boundary we use.
"""
exahype2_project.set_global_simulation_parameters(
    dimensions            = 2,
    size                  = [2.0,  2.0],
    offset                = [-1.0, -1.0],
    min_end_time          = 0.5
    max_end_time          = 0.5
    first_plot_time_stamp = 0.0,
    time_in_between_plots = 0.1,
    periodic_BC           = [False, False],
)

In [ ]:
"""
Finally, we specify some optimizations for parallel runs and compile the project.
"""
exahype2_project.set_load_balancing(
    "toolbox::loadbalancing::strategies::SpreadOutOnceGridStagnates",
    "new ::exahype2::LoadBalancingConfiguration()"
)

exahype2_project.set_Peano4_installation("", peano4.output.CompileMode.Release)

peano4_project = exahype2_project.generate_Peano4_project()
peano4_project.output.makefile.add_CXX_flag("-w")
peano4_project.build(make=True, make_clean_first=True, throw_away_data_after_build=True)

In [ ]:
!OMP_NUM_THREADS=4 ./DamBreak > output.txt 2>&1

In [ ]:
!python $PEANO_ROOT/python/peano4/visualisation/render.py solutions/solution-SWESolver.peano-patch-file